In [13]:
import requests
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# For normal

In [14]:
from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport

# Select your transport with a defined url endpoint
transport = AIOHTTPTransport(url="https://countries.trevorblades.com/")

# Create a GraphQL client using the defined transport
client = Client(transport=transport, fetch_schema_from_transport=True)

# Provide a GraphQL query


# Execute the query on the transport
# result = client.execute(query)
# print(result)


# Async for ipython

In [15]:
import asyncio

from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport

transport = AIOHTTPTransport(url="https://api.thegraph.com/subgraphs/name/connext/nxtp-matic")

## Graph Queries

### Transactions from User

In [16]:
query = gql(
    """
query GetSenderTransactions($userId: String!, $sendingChainId: BigInt!, $status: TransactionStatus) {
  transactions(
    where: {user: $userId, status: $status, sendingChainId: $sendingChainId}
    orderBy: preparedBlockNumber
    orderDirection: desc
  ) {
    id
    status
    chainId
    preparedTimestamp
    user {
      id
    }
    router {
      id
    }
    receivingChainTxManagerAddress
    sendingAssetId
    receivingAssetId
    sendingChainFallback
    receivingAddress
    callTo
    sendingChainId
    receivingChainId
    callDataHash
    transactionId
    amount
    expiry
    preparedBlockNumber
    bidSignature
    prepareCaller
    fulfillCaller
    cancelCaller
    prepareTransactionHash
    fulfillTransactionHash
    cancelTransactionHash
  }
}
"""
)

params = {
        "sendingChainId": 137,
        "userId": "0x4752a32843b661fedc525c979aaa01fa54d31366",
        "status": "Fulfilled"
    }

In [19]:
query = gql(
    """
query fetchAllTransactions($batch: Int!) {
  transactions (
    first:1000, skip: $batch,
    orderBy: preparedBlockNumber
    orderDirection: asc
  ){
    id
    status
    chainId
    preparedTimestamp
    user {
      id
    }
    router {
      id
    }
    receivingChainTxManagerAddress
    sendingAssetId
    receivingAssetId
    sendingChainFallback
    receivingAddress
    callTo
    sendingChainId
    receivingChainId
    callDataHash
    transactionId
    amount
    expiry
    preparedBlockNumber
    bidSignature
    prepareCaller
    fulfillCaller
    cancelCaller
    prepareTransactionHash
    fulfillTransactionHash
    cancelTransactionHash
  }
}
"""
)

params = {
        "batch": 0
    }

## Querying and Fetching data

In [20]:
matic_txns = pd.DataFrame(columns=['amount', 'bidSignature', 'callDataHash', 'callTo', 'cancelCaller', 'cancelTransactionHash', 'chainId', 'expiry', 'fulfillCaller', 'fulfillTransactionHash', 'id', 'prepareCaller', 'prepareTransactionHash', 'preparedBlockNumber', 'preparedTimestamp', 'receivingAddress', 'receivingAssetId', 'receivingChainId', 'receivingChainTxManagerAddress', 'router', 'sendingAssetId', 'sendingChainFallback', 'sendingChainId', 'status', 'transactionId', 'user'])
matic_transport = AIOHTTPTransport(url="https://api.thegraph.com/subgraphs/name/connext/nxtp-matic")

bsc_txns = pd.DataFrame(columns=['amount', 'bidSignature', 'callDataHash', 'callTo', 'cancelCaller', 'cancelTransactionHash', 'chainId', 'expiry', 'fulfillCaller', 'fulfillTransactionHash', 'id', 'prepareCaller', 'prepareTransactionHash', 'preparedBlockNumber', 'preparedTimestamp', 'receivingAddress', 'receivingAssetId', 'receivingChainId', 'receivingChainTxManagerAddress', 'router', 'sendingAssetId', 'sendingChainFallback', 'sendingChainId', 'status', 'transactionId', 'user'])
bsc_transport = AIOHTTPTransport(url="https://api.thegraph.com/subgraphs/name/connext/nxtp-bsc")

xdai_txns = pd.DataFrame(columns=['amount', 'bidSignature', 'callDataHash', 'callTo', 'cancelCaller', 'cancelTransactionHash', 'chainId', 'expiry', 'fulfillCaller', 'fulfillTransactionHash', 'id', 'prepareCaller', 'prepareTransactionHash', 'preparedBlockNumber', 'preparedTimestamp', 'receivingAddress', 'receivingAssetId', 'receivingChainId', 'receivingChainTxManagerAddress', 'router', 'sendingAssetId', 'sendingChainFallback', 'sendingChainId', 'status', 'transactionId', 'user'])
xdai_transport = AIOHTTPTransport(url="https://api.thegraph.com/subgraphs/name/connext/nxtp-xdai")

fantom_txns = pd.DataFrame(columns=['amount', 'bidSignature', 'callDataHash', 'callTo', 'cancelCaller', 'cancelTransactionHash', 'chainId', 'expiry', 'fulfillCaller', 'fulfillTransactionHash', 'id', 'prepareCaller', 'prepareTransactionHash', 'preparedBlockNumber', 'preparedTimestamp', 'receivingAddress', 'receivingAssetId', 'receivingChainId', 'receivingChainTxManagerAddress', 'router', 'sendingAssetId', 'sendingChainFallback', 'sendingChainId', 'status', 'transactionId', 'user'])
fantom_transport = AIOHTTPTransport(url="https://api.thegraph.com/subgraphs/name/connext/nxtp-fantom")

In [21]:
async def graph_query(query, params, transport, transactions):

    # Using `async with` on the client will start a connection on the transport
    # and provide a `session` variable to execute queries on this connection
    async with Client(
        transport=transport, fetch_schema_from_transport=True,
    ) as session:

        # Execute single query
        for batch in range(42): #Just a random no. It has to be just > 6. 
            result = await session.execute(query, variable_values=params)
            for tr in result["transactions"]:
                list_values = list(tr.values())
                list_values[19] = list_values[19]["id"]
                list_values[25] = list_values[25]["id"]
                transactions.loc[len(transactions.index)] = list_values
            if len(result["transactions"]) < 1000:
                break
            params['batch']+=1000
            print(params["batch"], end="-")
        print("")
        return result

In [22]:
params["batch"] = 0
await graph_query(query, params, matic_transport, matic_txns)
params["batch"] = 0
await graph_query(query, params, bsc_transport, bsc_txns)
params["batch"] = 0
await graph_query(query, params, xdai_transport, xdai_txns)
params["batch"] = 0
await graph_query(query, params, fantom_transport, fantom_txns)
params["batch"] = 0

1000-2000-3000-4000-
1000-2000-3000-

1000-


### Chain Stats

In [24]:
matic_txns[matic_txns.sendingChainId=="137"]["status"].value_counts()

Fulfilled    2981
Cancelled     174
Prepared       20
Name: status, dtype: int64

In [25]:
bsc_txns[bsc_txns.sendingChainId=="56"]["status"].value_counts()

Fulfilled    1106
Cancelled     183
Prepared       47
Name: status, dtype: int64

In [26]:
xdai_txns[xdai_txns.sendingChainId=="100"].shape

(325, 26)

In [27]:
fantom_txns[fantom_txns.sendingChainId=="250"].shape

(334, 26)

# Analysing data

## Data Cleaning

### Combining all data

In [67]:
chain_asset_data = {
   "Ethereum":{
      "0x0000000000000000000000000000000000000000":{
         "token":"ETH",
         "decimals":18
      },
      "0x7d1afa7b718fb893db30a3abc0cfc608aacfebb0":{
         "token":"MATIC",
         "decimals":18
      },
      "0xb8c77482e45f1f44de1745f52c74426c631bdd52":{
         "token":"BNB",
         "decimals":18
      },
      "0x6b175474e89094c44da98b954eedeac495271d0f":{
         "token":"DAI",
         "decimals":18
      },
      "0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48":{
         "token":"USDC",
         "decimals":6
      },
      "0xdac17f958d2ee523a2206206994597c13d831ec7":{
         "token":"USDT",
         "decimals":6
      }
   },
   "BSC":{
      "0x2170ed0880ac9a755fd29b2688956bd959f933f8":{
         "token":"ETH",
         "decimals":18
      },
      "0xcc42724c6683b7e57334c4e856f4c9965ed682bd":{
         "token":"MATIC",
         "decimals":18
      },
      "0x0000000000000000000000000000000000000000":{
         "token":"BNB",
         "decimals":18
      },
      "0x1af3f329e8be154074d8769d1ffa4ee058b1dbc3":{
         "token":"DAI",
         "decimals":18
      },
      "0x8ac76a51cc950d9822d68b83fe1ad97b32cd580d":{
         "token":"USDC",
         "decimals":18
      },
      "0x55d398326f99059ff775485246999027b3197955":{
         "token":"USDT",
         "decimals":18
      }
   },
   "Polygon":{
      "0xfd8ee443ab7be5b1522a1c020c097cff1ddc1209":{
         "token":"ETH",
         "decimals":18
      },
      "0x0000000000000000000000000000000000000000":{
         "token":"MATIC",
         "decimals":18
      },
      "0xa649325aa7c5093d12d6f98eb4378deae68ce23f":{
         "token":"BNB",
         "decimals":18
      },
      "0x8f3cf7ad23cd3cadbd9735aff958023239c6a063":{
         "token":"DAI",
         "decimals":18
      },
      "0x2791bca1f2de4661ed88a30c99a7a9449aa84174":{
         "token":"USDC",
         "decimals":6
      },
      "0xc2132d05d31c914a87c6611c10748aeb04b58e8f":{
         "token":"USDT",
         "decimals":6
      }
   },
   "xDai":{
      "0xa5c7cb68cd81640d40c85b2e5ec9e4bb55be0214":{
         "token":"ETH",
         "decimals":18
      },
      "0x7122d7661c4564b7c6cd4878b06766489a6028a2":{
         "token":"MATIC",
         "decimals":18
      },
      "0xca8d20f3e0144a72c6b5d576e9bd3fd8557e2b04":{
         "token":"BNB",
         "decimals":18
      },
      "0x0000000000000000000000000000000000000000":{
         "token":"DAI",
         "decimals":18
      },
      "0xddafbb505ad214d7b80b1f830fccc89b60fb7a83":{
         "token":"USDC",
         "decimals":6
      },
      "0x4ecaba5870353805a9f068101a40e0f32ed605c6":{
         "token":"USDT",
         "decimals":6
      }
   },
   "Fantom":{
      "0x658b0c7613e890ee50b8c4bc6a3f41ef411208ad":{
         "token":"ETH",
         "decimals":18
      },
      "0x8d11ec38a3eb5e956b052f67da8bdc9bef8abf3e":{
         "token":"DAI",
         "decimals":18
      },
      "0x04068da6c83afcfa0e13ba15a6696662335d5b75":{
         "token":"USDC",
         "decimals":6
      },
      "0x049d68029688eabf473097a2fc38ef61633a3c7a":{
         "token":"USDT",
         "decimals":6
      }
   }
}

In [68]:
matic_txns["chain"] = "Polygon"

bsc_txns["chain"] = "BSC"

xdai_txns["chain"] = "xDai"

fantom_txns["chain"] = "Fantom"

In [87]:
two_sided_txns = pd.concat([matic_txns,bsc_txns,xdai_txns,fantom_txns])

In [98]:
two_sided_txns.shape

(10258, 27)

In [114]:
two_sided_txns["txn_type"] = two_sided_txns.apply(lambda x: "single" if x["sendingChainId"] == x["chainId"] else "repeat", axis=1)

In [125]:
one_sided_txns = two_sided_txns[two_sided_txns["txn_type"]=="single"].copy(deep=True)

In [126]:
one_sided_txns.shape

(5170, 29)

### Asset Movement

In [118]:
chain_mapping = {"56": "BSC", "137": "Polygon", "250": "Fantom", "100": "xDai"}

In [119]:
def transacting_chains(row):
    val = chain_mapping[row["sendingChainId"]] + " -> " + chain_mapping[row["receivingChainId"]]
    return val

In [127]:
one_sided_txns['asset_movement'] = one_sided_txns.apply(transacting_chains, axis=1)
one_sided_txns['asset_movement'].value_counts()

Polygon -> BSC       2161
BSC -> Polygon       1076
Polygon -> Fantom     763
Polygon -> xDai       251
BSC -> Fantom         213
xDai -> Polygon       210
Fantom -> Polygon     191
Fantom -> BSC         114
xDai -> Fantom         60
xDai -> BSC            55
BSC -> xDai            47
Fantom -> xDai         29
Name: asset_movement, dtype: int64

In [128]:
one_sided_txns[one_sided_txns["status"]=="Fulfilled"]['asset_movement'].value_counts()

Polygon -> BSC       2044
BSC -> Polygon        892
Polygon -> Fantom     690
Polygon -> xDai       247
xDai -> Polygon       199
Fantom -> Polygon     170
BSC -> Fantom         167
Fantom -> BSC         110
xDai -> Fantom         57
xDai -> BSC            54
BSC -> xDai            47
Fantom -> xDai         28
Name: asset_movement, dtype: int64

In [97]:
one_sided_txns[one_sided_txns["status"]=="Fulfilled"]['asset_movement'].count()

4705

### Mapping Assets to Txns

In [143]:
def asset_token_mapper(row):
    chain_asset_dict = chain_asset_data[row["chain"]]
    asset = chain_asset_dict[row["sendingAssetId"]]
    return asset["token"]

In [145]:
def asset_decimal_mapper(row):
    chain_asset_dict = chain_asset_data[row["chain"]]
    asset = chain_asset_dict[row["sendingAssetId"]]
    return asset["decimals"]

In [150]:
10**5

100000

In [163]:
def dollar_amount(row):
    dollar_value = int(row["amount"]) / 10**row["decimals"]
    return dollar_value

In [166]:
for row in one_sided_txns.iterrows():
    print(row[1]["decimals"], end=",")
    print(row[1]["amount"], end=" = ")
    print(dollar_amount(row[1]))
    if row[0] == 12:
        break

6,1000000 = 1.0
6,1000000 = 1.0
6,1000000 = 1.0
6,100000000 = 100.0
6,10000 = 0.01
6,10000000 = 10.0
6,20000 = 0.02


In [147]:
one_sided_txns["asset_token"] = one_sided_txns.apply(asset_token_mapper, axis=1)

In [148]:
one_sided_txns["decimals"] = one_sided_txns.apply(asset_decimal_mapper, axis=1)

In [167]:
one_sided_txns["dollar_amount"] = one_sided_txns.apply(dollar_amount, axis=1)

In [168]:
one_sided_txns.dollar_amount.sum()

6490842.980980501

In [216]:
one_sided_txns[one_sided_txns['status']=="Fulfilled"]["dollar_amount"].sum()

5767142.284557633

In [219]:
x = one_sided_txns[one_sided_txns['status']=="Fulfilled"].groupby("receivingChainId", axis=0).sum()
x

,decimals,dollar_amount
receivingChainId,,
100,4392,1.667718e+05
137,20118,2.468027e+06
250,10020,1.564222e+06
56,15828,1.568122e+06


In [220]:
for row in x.iterrows():
    print(row[0], end=" = ")
    print(row[1][1])

100 = 166771.80522999287
137 = 2468026.8525046837
250 = 1564221.643998964
56 = 1568121.9828239896


In [218]:
for row in x.iterrows():
    print(row[0], end=" = ")
    print(row[1][1])

BSC = 1585771.223553348
Fantom = 1246014.0917421093
Polygon = 2763072.4851431698
xDai = 172284.48411900012


In [176]:
one_sided_txns.columns

Index(['amount', 'bidSignature', 'callDataHash', 'callTo', 'cancelCaller',
       'cancelTransactionHash', 'chainId', 'expiry', 'fulfillCaller',
       'fulfillTransactionHash', 'id', 'prepareCaller',
       'prepareTransactionHash', 'preparedBlockNumber', 'preparedTimestamp',
       'receivingAddress', 'receivingAssetId', 'receivingChainId',
       'receivingChainTxManagerAddress', 'router', 'sendingAssetId',
       'sendingChainFallback', 'sendingChainId', 'status', 'transactionId',
       'user', 'chain', 'txn_type', 'asset_movement', 'asset_token',
       'decimals', 'dollar_amount'],
      dtype='object')

In [187]:
one_sided_txns[one_sided_txns.dollar_amount==100685.0]["transactionId"].values

array(['0xf24169c8557f87f3b0ad7ede8218a9a0551a72c43751fbd75130e5fe1bc247ed'],
      dtype=object)

In [183]:
one_sided_txns[one_sided_txns.dollar_amount==100685.0]["asset_movement"].values

array(['Fantom -> Polygon'], dtype=object)

In [186]:
one_sided_txns[one_sided_txns.dollar_amount==100685.0]["fulfillTransactionHash"].values

array(['0xcf664975c10de3a0e3781493d5b8c2d7ca32ee5d4880269458ad8be92705355b'],
      dtype=object)

In [185]:
one_sided_txns[one_sided_txns.dollar_amount==100685.0]

,amount,bidSignature,callDataHash,callTo,cancelCaller,cancelTransactionHash,chainId,expiry,fulfillCaller,fulfillTransactionHash,id,prepareCaller,prepareTransactionHash,preparedBlockNumber,preparedTimestamp,receivingAddress,receivingAssetId,receivingChainId,receivingChainTxManagerAddress,router,sendingAssetId,sendingChainFallback,sendingChainId,status,transactionId,user,chain,txn_type,asset_movement,asset_token,decimals,dollar_amount
156,100685000000,0x566d01c81fe7257d081568fada815d5d6e708cebc8ed...,0xc5d2460186f7233c927e7db2dcc703c0e500b653ca82...,0x0000000000000000000000000000000000000000,None,None,250,1630877968,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0xcf664975c10de3a0e3781493d5b8c2d7ca32ee5d4880...,0xf24169c8557f87f3b0ad7ede8218a9a0551a72c43751...,0xf7f0cfc3772d29d4cc1482a2acb7be16a85a2223,0xf2533ae556052fa4357bfc0e314412f411e44aa4f40b...,16187471,1630618824,0xf7f0cfc3772d29d4cc1482a2acb7be16a85a2223,0xc2132d05d31c914a87c6611c10748aeb04b58e8f,137,0x31efc4aeaa7c39e54a33fdc3c46ee2bd70ae0a09,0xe1dd7b28ab085978ead65aff92b22b6322a96422,0x049d68029688eabf473097a2fc38ef61633a3c7a,0xf7f0cfc3772d29d4cc1482a2acb7be16a85a2223,250,Fulfilled,0xf24169c8557f87f3b0ad7ede8218a9a0551a72c43751...,0xf7f0cfc3772d29d4cc1482a2acb7be16a85a2223,Fantom,single,Fantom -> Polygon,USDT,6,100685.0
